In [ ]:
import sqlite3
from tqdm import tqdm
import vk
import time

In [ ]:
#open vk session
with open('token') as f:
    TOKEN = f.read()
session = vk.Session(access_token=TOKEN)
vkapi = vk.API(session)

#constants
SELF_ID = 7110301
SLEEP_TIME = 0.3

In [ ]:
#database
conn = sqlite3.connect('users.db')
c = conn.cursor()

c.execute('CREATE TABLE IF NOT EXISTS uids (uid STRING)');
c.execute('CREATE TABLE IF NOT EXISTS users (uid STRING, first_name STRING, last_name STRING, sex STRING, city STRING, home_city STRING)');
c.execute('CREATE TABLE IF NOT EXISTS posts (owner_id STRING, date STRING, text STRING)');

In [ ]:
%%time
users = vkapi('friends.get', user_id=SELF_ID, v=5.73)['items']
users2 = []
for user in users:
    try:
        users2 += vkapi('friends.get', user_id = user, v=5.73)['items']
        time.sleep(SLEEP_TIME)
    except:
        continue
    if len(users2) > 300: #will get ~ 400 000 users without break
        print("limit reached".upper())
        break
users += users2
users = set(users)

users = [7110301,16498689, 7479298, 139411465, 475146, 19914761, 183230474, 5324819, 
         1581076, 17227799, 5111832, 5808166]

c.executemany('INSERT INTO uids VALUES (?)', [(u,) for u in users])
conn.commit()

In [ ]:
%%time
for user in tqdm(users):
    try:
        uinfo = vkapi('users.get', user_id = user, fields = ['sex', 'city', 'home_town'], v=5.73)
        uinfo[0]['city'] = uinfo[0]['city']['title']
        time.sleep(SLEEP_TIME)
        for u in uinfo:
            data = tuple(v for v in u.values())
            if len(data) < 6: data = data + (0,)
            c.execute('INSERT INTO users VALUES (?,?,?,?,?,?)', data)
            conn.commit()
    except:
        continue

In [ ]:
%%time
def get_post_count(friend):
    posts = vkapi('wall.get', owner_id=friend, count=1, filter='owner', v=5.73)
    return posts['count']

drop = 0
for user in tqdm(users, desc='Downloading posts'):
    try:
        current_post_count = get_post_count(user)
        time.sleep(SLEEP_TIME)
        if (current_post_count > 7000) or (current_post_count < 100):
            continue
        current_posts = []
        resid = current_post_count
        offset = 0
#         while resid > 0:
        response = vkapi('wall.get', owner_id=user, count=100, filter='owner', 
                         offset=offset, v=5.73)['items']
        time.sleep(SLEEP_TIME)
        current_posts += response
#             resid -= 100
#             offset += 100
        chunk = []
        for p in current_posts:
            if type(p) == dict:
                chunk.append((p['owner_id'], p['date'], p['text']))
        c.executemany('INSERT INTO posts VALUES (?,?,?)', chunk)
        conn.commit()
    except:
        drop += 1
        continue

In [ ]:
c.close()
conn.close()